In [1]:
import os
import glob

import xarray as xr
import numpy as np

In [2]:
data_m = '/home/hannasv/Desktop/lagrings/ERA5_monthly/'

In [3]:
test = '2005-05'

# Data exploration we want to study :
* Correlation pixelwise to pressure
* Everything as a mean over time 
* Hvordan dele opp dataen slik at det kan leses inn best mulig. 
* Hugo minnet sette begrensninger for AR - models med invers matrise?
* Hvor mye data har jeg egentlig?

In [4]:
# Hyperparameters useful stuff - 
num_epochs = 4
batch_size = 32
#learning_rate = 0.001

#DATA_PATH = '/uio/lagringshotell/geofag/students/metos/hannasv/era_interim_data/'
#MODEL_STORE_PATH = '/uio/lagringshotell/geofag/students/metos/hannasv/era_interim_data/storedModel/'

DATA_PATH = '/uio/lagringshotell/geofag/students/metos/hannasv/era_interim_data/'
MODEL_STORE_PATH = '/uio/lagringshotell/geofag/students/metos/hannasv/era_interim_data/storedModel/'

In [5]:
"%2.2d" %3

'03'

In [6]:
'2012_02' < '2012_01'

False

In [ ]:
def get_files_excluding_gap(start_gap, stop_gap):
    pass

In [16]:
def get_files(start, stop):
    """ Get files in requested period.
    Insert test on start > stop. 
    
    Returns:
        List of files to be read into 
    
    """
    y, m = start.split('_')
    start_y = int(y)
    start_m = int(m)
    
    y, m = stop.split('_')
    stop_y = int(y)
    stop_m = int(m)

    years = np.arange(start_y, stop_y+1)
    months = np.arange(1, 13)  
    search_str = [start, stop]
    
    for y in years:
        for m in months:
            m = "%2.2d" %m
            tmp = '{}_{}'.format(y, m)
            if tmp > start and tmp < stop:
                search_str.append(tmp)   
    
    storage = '/home/hannasv/Desktop/lagrings/ERA5_monthly/'
    
    files = []
    for ss in search_str:
        tmp_files = glob.glob(os.path.join( storage, '*{}*.nc'.format(ss)  ))  
        files += tmp_files
    return files

In [17]:
# get_files('2012_03', '2014_01')

In [50]:
# Loader of AR 
def load_data_AR(start, stop, lat, lon, num_ts = 0, bias = True, validation_split = 0.0):
    """ Prepares data for 
    
    start, stop : str
        year_month. Limited to montly values.
        
    lat, lon : float
        coordinate information
        
    bias : bool 
        Default 1
        
    validation_split : float
        Number between 0 and 1. 
        Intended to make it similar to keras input data. Will it make it easier to train networks though...?
    """
    if validation_split > 0:
        raise NotImplementedError('Comming soon ... Make sure that you actually need this?')
        
    # Search for list of files in the range start stop.
    # Returns list of strings you can merge to a dataset 
    
    files = get_files(start, stop)
    datasets = [ xr.open_dataset(fil).sel(latitude = lat, longitude = lon) for fil in files]
    data = xr.merge(datasets) 
    
    n     = len(data.time.values)
    #n_lat = len(data.latitude.values)
    #n_lon = len(data.longitude.values)
    
    q   = data.q.values
    t2m = data.t2m.values
    r   = data.r.values
    sp  = data.sp.values
    
    tcc = data.tcc.values
    
    if bias:
        num_vars = 4+1+num_ts
    else:
        raise NotImplementedError('Coming soon')
        num_vars = 4 + num_ts
        
    num_samples = n - num_ts
    X = np.zeros((num_samples, num_vars))
    
    X[:, 0] = 1
    X[:, 1] = t2m[:n-num_ts]
    X[:, 2] = r[:n-num_ts]
    X[:, 3] = sp[:n-num_ts]
    X[:, 4] = q[:n-num_ts]
    
    y = tcc[:n-num_ts, np.newaxis]
    
    for i in range(1, num_ts+1):
        if num_ts - i != 0:
            X[:, 4+i] = tcc[i:num_ts-i]
        else:
            X[:, 4+i] = tcc[i:]
    
    # Now drop rows contaning nans.
    no_nans =  X[~np.isnan(X.any(axis=1))]
    return no_nans, y

In [46]:
start='2012_02'
stop='2012_04'

lat=30
lon = 15

In [51]:
X, y = load_data_AR(start, stop, lat, lon, num_ts = 1, bias = True, validation_split = 0.0)

/home/hannasv/anaconda3/envs/sciclouds/lib/python3.6/site-packages/xarray/coding/times.py:207: FutureWarning: the 'box' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'box'
  result = pd.to_timedelta(num_timedeltas, unit=units, box=False)
/home/hannasv/anaconda3/envs/sciclouds/lib/python3.6/site-packages/xarray/coding/times.py:207: FutureWarning: the 'box' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'box'
  result = pd.to_timedelta(num_timedeltas, unit=units, box=False)
/home/hannasv/anaconda3/envs/sciclouds/lib/python3.6/site-packages/xarray/coding/times.py:207: FutureWarning: the 'box' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'box'
  result = pd.to_timedelta(num_timedeltas, unit=units, box=False)
/home/hannasv/anaconda3/envs/sciclouds/lib/python3.6/site-packages/xarray/coding/times.py:207: FutureWarning: the 'box' keyw

In [28]:
start, stop, lat, lon, 

('2012_02', '2012_04', 30, 15)

In [13]:
#MergeError: conflicting values for variable 'valid_time' on objects to be combined:

In [52]:
X.shape, y.shape

((2159, 6), (2159, 1))

# Make a plot containing the sum of all nans over all gridded times¨

In [ ]:
files = glob.glob(os.path.join( data_m, "*2007_08*.nc" ))

In [ ]:
len(files)

In [ ]:
test = xr.open_dataset(files[0])

In [ ]:
test

In [ ]:
datasets = [ xr.open_dataset(fil) for fil in files  ]

In [ ]:
for i in range(5):
    print(datasets[i])

In [ ]:
data = xr.merge(datasets, 'no_conflicts', 'outer')

In [ ]:
data

In [ ]:
dropped = data.dropna(dim='time')